In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

path = r'/Users/dewiindahpurnama_/Documents/Python/property-data/Property Listing Data.xlsx'
df = pd.read_excel(path, dtype=str, sheet_name='Sheet1')
df

,Record Id,Zip Code,Property Name,Seller Account Owner.id,Seller Account Name,Phone,Agents,Link,Scraped_Price
0,zcrm_6965175000001846208,NaN,NaN,zcrm_6965175000000598187,Rick Leenheer,NaN,NaN,NaN,NaN
1,zcrm_6965175000002165378,NaN,NaN,zcrm_6965175000000598187,Coco,+6281011094,NaN,NaN,NaN
2,zcrm_6965175000003577206,SC1038,Modern 4-Bedroom Sanctuary in Pejeng’s Cultura...,zcrm_6965175000000667009,Victor Remy,NaN,Suci,https://matapropertybali.com/property/modern-4...,"9,380,000,000 IDR"
3,zcrm_6965175000003577207,SC1017,"Modern 3-Bedroom Villa in Singakerta for Sale,...",zcrm_6965175000000667009,Cedric Henry Levi,NaN,Suci,https://matapropertybali.com/property/modern-3...,"5,860,000,000 IDR"
4,zcrm_6965175000003577208,SC1141,"Serene 3-Bedroom Jungle Villa in Tegalalang, Ubud",zcrm_6965175000000667009,BSH,NaN,Suci,https://matapropertybali.com/property/villa-wi...,"6,360,000,000 IDR"
...,...,...,...,...,...,...,...,...,...
527,zcrm_6965175000003616239,SC554,A breathtaking plot of land in Tegalalang for ...,zcrm_6965175000000667009,CP: Juli Optic,+62 831-1616-7305,Suci,https://matapropertybali.com/property/a-breath...,"5,000,000 IDR/are /year"
528,zcrm_6965175000003616240,SC506,"Stylish Apartments in Ubud for Sale, Bali",zcrm_6965175000000667009,Balix Development,NaN,Suci,https://matapropertybali.com/property/stylish-...,"2,650,000,000 IDR"
529,zcrm_6965175000003616241,SC457,Stunning 3-Bedroom Villa with Ocean View in Ce...,zcrm_6965175000000667009,CP: Lanny,+62 813-1920-0900,Suci,https://matapropertybali.com/property/discover...,"5,500,000,000 IDR"
530,zcrm_6965175000003616242,SC450,Newly Built 2 bedroom Villa in Nusadua,zcrm_6965175000000667009,JSP Development,+62 812-3661-9575,Suci,https://matapropertybali.com/property/newly-bu...,"2,700,000,000 IDR"


In [7]:
# 1. Identify which rows are duplicates (excluding the first one)
# keep='first' means the first occurrence stays False (Unique), 
# and all subsequent matches become True.
is_duplicate = df.duplicated(subset=['Seller Account Name', 'Phone'], keep='first')

# 2. Identify the "Main Leads"
# These are rows that have matches elsewhere but are the first instance.
# We find them by looking for rows that are duplicated when keep=False,
# but NOT duplicated when keep='first'.
all_matches = df.duplicated(subset=['Seller Account Name', 'Phone'], keep=False)
is_main_lead = all_matches & ~is_duplicate

# 3. Apply the Labels
df['Leads_Status'] = 'Unique' # Default for records with no matches
df.loc[is_main_lead, 'Leads_Status'] = 'Main Lead'
df.loc[is_duplicate, 'Leads_Status'] = 'Duplicate'

In [8]:
df

,Record Id,Zip Code,Property Name,Seller Account Owner.id,Seller Account Name,Phone,Agents,Link,Scraped_Price,Duplicate_Status,Leads_Status
0,zcrm_6965175000001846208,NaN,NaN,zcrm_6965175000000598187,Rick Leenheer,NaN,NaN,NaN,NaN,Unique,Unique
1,zcrm_6965175000002165378,NaN,NaN,zcrm_6965175000000598187,Coco,+6281011094,NaN,NaN,NaN,Unique,Unique
2,zcrm_6965175000003577206,SC1038,Modern 4-Bedroom Sanctuary in Pejeng’s Cultura...,zcrm_6965175000000667009,Victor Remy,NaN,Suci,https://matapropertybali.com/property/modern-4...,"9,380,000,000 IDR",Unique,Unique
3,zcrm_6965175000003577207,SC1017,"Modern 3-Bedroom Villa in Singakerta for Sale,...",zcrm_6965175000000667009,Cedric Henry Levi,NaN,Suci,https://matapropertybali.com/property/modern-3...,"5,860,000,000 IDR",Unique,Unique
4,zcrm_6965175000003577208,SC1141,"Serene 3-Bedroom Jungle Villa in Tegalalang, Ubud",zcrm_6965175000000667009,BSH,NaN,Suci,https://matapropertybali.com/property/villa-wi...,"6,360,000,000 IDR",Duplicate,Main Lead
...,...,...,...,...,...,...,...,...,...,...,...
527,zcrm_6965175000003616239,SC554,A breathtaking plot of land in Tegalalang for ...,zcrm_6965175000000667009,CP: Juli Optic,+62 831-1616-7305,Suci,https://matapropertybali.com/property/a-breath...,"5,000,000 IDR/are /year",Unique,Unique
528,zcrm_6965175000003616240,SC506,"Stylish Apartments in Ubud for Sale, Bali",zcrm_6965175000000667009,Balix Development,NaN,Suci,https://matapropertybali.com/property/stylish-...,"2,650,000,000 IDR",Duplicate,Duplicate
529,zcrm_6965175000003616241,SC457,Stunning 3-Bedroom Villa with Ocean View in Ce...,zcrm_6965175000000667009,CP: Lanny,+62 813-1920-0900,Suci,https://matapropertybali.com/property/discover...,"5,500,000,000 IDR",Unique,Unique
530,zcrm_6965175000003616242,SC450,Newly Built 2 bedroom Villa in Nusadua,zcrm_6965175000000667009,JSP Development,+62 812-3661-9575,Suci,https://matapropertybali.com/property/newly-bu...,"2,700,000,000 IDR",Unique,Unique


In [13]:
output_path = r'/Users/dewiindahpurnama_/Documents/Python/property-data/processed_leads_zoho.xlsx'
df.to_excel(output_path, index=False)